# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from tqdm import tqdm
import time
import random

BATCH_SIZE = 128
NUM_EPOCHS = 10
learning_rate = 0.01

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [2]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [3]:
class SimpleNet(nn.Module):
# TODO:define model
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.conv1 = nn.Sequential(      # input 1*28*28
            nn.Conv2d(1, 6, 5, 1, 2),    # padding=2
            nn.ReLU(),                   # input 6*28*28
            nn.MaxPool2d(kernel_size=2, stride=2), # output 6*14*14
        )
        self.conv2 = nn.Sequential(      # input 6*14*14
            nn.Conv2d(6, 16, 5),
            nn.ReLU(),                   # input 16*10*10
            nn.MaxPool2d(2, 2),          # output 16*5*5
        )
        self.fc1 = nn.Sequential(
            nn.Linear(16 * 5 * 5, 120),
            nn.ReLU(),
        )
        self.fc2 = nn.Sequential(
            nn.Linear(120, 84),
            nn.ReLU()
        )
        self.fc3 = nn.Linear(84, 10)
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size()[0], -1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

model = SimpleNet()

# TODO:define loss function and optimiter
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

Next, we can start to train and evaluate!

In [4]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    train_loss = 0
    cnt = 0

    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize    
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        cnt += 1
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset
    with torch.no_grad():
        acc = 0
        total = 0
        for images, labels in tqdm(test_loader):
            outputs = model(images)
            predicted = torch.max(outputs.data, 1)[1]
            total += labels.size(0)
            acc += (predicted == labels).sum()

    print('\tepoch%d train loss: %.03f test accuracy:%d%%'
                        % (epoch + 1, train_loss / cnt, 100 * acc / total))

100%|██████████| 78/78 [00:01<00:00, 48.47it/s]	epoch10 train loss: 0.021 test accuracy:98%



#### Q5:
Please print the training and testing accuracy.

In [5]:
# train and evaluate
for epoch in range(1):
    train_acc = 0
    cnt = 0
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize    
        optimizer.zero_grad()
        outputs = model(images)
        predicted = torch.max(outputs.data, 1)[1]
        cnt += labels.size(0)
        train_acc += (predicted == labels).sum()
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset
    with torch.no_grad():
        test_acc = 0
        total = 0
        for images, labels in tqdm(test_loader):
            outputs = model(images)
            predicted = torch.max(outputs.data, 1)[1]
            total += labels.size(0)
            test_acc += (predicted == labels).sum()

    print('\ttrain accuracy: %d%% test accuracy:%d%%'
                        % (100 * train_acc / cnt, 100 * test_acc / total))

100%|██████████| 78/78 [00:01<00:00, 48.87it/s]	train accuracy: 99% test accuracy:98%

